In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb

# ----------------------------
# 1. 데이터 불러오기
# ----------------------------
df = pd.read_csv("spotify_songs.csv")

print(df.head(5))

                 track_id                                         track_name  \
0  6f807x0ima9a1j3VPbc7VN  I Don't Care (with Justin Bieber) - Loud Luxur...   
1  0r7CVbZTWZgbTCYdfa2P31                    Memories - Dillon Francis Remix   
2  1z1Hg7Vb0AhHDiEmnDE79l                    All the Time - Don Diablo Remix   
3  75FpbthrwQmzHlBJLuGdC7                  Call You Mine - Keanu Silva Remix   
4  1e8PAfcKUYoKkxPhrHqw4x            Someone You Loved - Future Humans Remix   

       track_artist  track_popularity          track_album_id  \
0        Ed Sheeran                66  2oCs0DGTsRO98Gh5ZSl2Cx   
1          Maroon 5                67  63rPSO264uRjW1X5E6cWv6   
2      Zara Larsson                70  1HoSmj2eLcsrR0vE9gThr4   
3  The Chainsmokers                60  1nqYsOef1yKKuGOVchbsk6   
4     Lewis Capaldi                69  7m7vv9wlQ4i0LFuJiE2zsQ   

                                    track_album_name track_album_release_date  \
0  I Don't Care (with Justin Bieber) [Loud Luxu

In [17]:
df.isnull().sum()


track_id                    0
track_name                  5
track_artist                5
track_popularity            0
track_album_id              0
track_album_name            5
track_album_release_date    0
playlist_name               0
playlist_id                 0
playlist_genre              0
playlist_subgenre           0
danceability                0
energy                      0
key                         0
loudness                    0
mode                        0
speechiness                 0
acousticness                0
instrumentalness            0
liveness                    0
valence                     0
tempo                       0
duration_ms                 0
dtype: int64

In [19]:
for col in df.columns:
    if df[col].isnull().sum() > 0:
        print(f"\n===== {col} (결측치 포함 행) =====")
        display(df[df[col].isnull()][[col]])



===== track_name (결측치 포함 행) =====


,track_name
8151,NaN
9282,NaN
9283,NaN
19568,NaN
19811,NaN



===== track_artist (결측치 포함 행) =====


,track_artist
8151,NaN
9282,NaN
9283,NaN
19568,NaN
19811,NaN



===== track_album_name (결측치 포함 행) =====


,track_album_name
8151,NaN
9282,NaN
9283,NaN
19568,NaN
19811,NaN


In [20]:
idx = [8151, 9282, 9283, 19568, 19811]
df.loc[idx]


,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
8151,69gRFGOWY9OMpFJgFol1u0,NaN,NaN,0,717UG2du6utFe7CdmpuUe3,NaN,2012-01-05,HIP&HOP,5DyJsJZOpMJh34WvUrQzMV,rap,...,6,-7.635,1,0.1760,0.0410,0.00000,0.1160,0.649,95.999,282707
9282,5cjecvX0CmC9gK0Laf5EMQ,NaN,NaN,0,3luHJEPw434tvNbme3SP8M,NaN,2017-12-01,GANGSTA Rap,5GA8GDo7RQC3JEanT81B3g,rap,...,11,-5.364,0,0.3190,0.0534,0.00000,0.5530,0.191,146.153,202235
9283,5TTzhRSWQS4Yu8xTgAuq6D,NaN,NaN,0,3luHJEPw434tvNbme3SP8M,NaN,2017-12-01,GANGSTA Rap,5GA8GDo7RQC3JEanT81B3g,rap,...,10,-5.907,0,0.3070,0.0963,0.00000,0.0888,0.505,86.839,206465
19568,3VKFip3OdAvv4OfNTgFWeQ,NaN,NaN,0,717UG2du6utFe7CdmpuUe3,NaN,2012-01-05,Reggaeton viejito🔥,0si5tw70PIgPkY1Eva6V8f,latin,...,11,-6.075,0,0.0366,0.0606,0.00653,0.1030,0.726,97.017,252773
19811,69gRFGOWY9OMpFJgFol1u0,NaN,NaN,0,717UG2du6utFe7CdmpuUe3,NaN,2012-01-05,latin hip hop,3nH8aytdqNeRbcRCg3dw9q,latin,...,6,-7.635,1,0.1760,0.0410,0.00000,0.1160,0.649,95.999,282707


In [10]:
# ============================================================
# 2. 결측치 처리
# ============================================================
df.fillna({'playlist_genre': df['playlist_genre'].mode()[0]}, inplace=True)
df.dropna(subset=['track_album_release_date'], inplace=True)

# ============================================================
# 3. 발매 연도 추출
# ============================================================
df['track_album_release_date'] = pd.to_datetime(df['track_album_release_date'], errors='coerce')
df['year'] = df['track_album_release_date'].dt.year

# ============================================================
# 4. 이상치 처리 (tempo, loudness clipping)
# ============================================================
for col in ['tempo', 'loudness']:
    upper = df[col].quantile(0.95)
    df[col] = np.where(df[col] > upper, upper, df[col])

# ============================================================
# 5. Feature 선택
# ============================================================
features = [
    'danceability', 'energy', 'valence', 'loudness', 'tempo',
    'acousticness', 'instrumentalness', 'playlist_genre', 'year'
]

X = df[features]
y = df['track_popularity']   # ← 수정됨 (popularity 아님!)

# ============================================================
# 6. One-Hot Encoding + Scaling (전처리 파이프라인)
# ============================================================
numeric_feats = X.select_dtypes(include=['int64','float64']).columns
categorical_feats = ['playlist_genre']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_feats),

        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_feats)
    ])

# ============================================================
# 7. Train/Test Split
# ============================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ============================================================
# 8. XGBoost 회귀 + Grid Search
# ============================================================
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

params = {
    'model__learning_rate': [0.05, 0.1],   # Pipeline 내부 param 네이밍 수정!
    'model__max_depth': [5, 7],
    'model__n_estimators': [200, 400]
}

pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', xgb_model)
])

grid = GridSearchCV(
    pipeline, params, cv=3, scoring='r2', n_jobs=-1
)

grid.fit(X_train, y_train)
best_model = grid.best_estimator_

print("Best Parameters:", grid.best_params_)

# ============================================================
# 9. 회귀 성능 평가
# ============================================================
pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, pred))
r2 = r2_score(y_test, pred)

print("RMSE:", rmse)
print("R2:", r2)

# ============================================================
# 10. 분류 모델 (인기곡 기준 60점 이상)
# ============================================================
df['label'] = (df['track_popularity'] >= 60).astype(int)   # ← 수정됨
y_cls = df['label']

# Train/Test 다시 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y_cls, test_size=0.2, random_state=42
)

clf = Pipeline([
    ('preprocess', preprocessor),
    ('model', GradientBoostingClassifier())
])

clf.fit(X_train, y_train)

pred = clf.predict(X_test)

print("Acc:", accuracy_score(y_test, pred))
print("F1:", f1_score(y_test, pred))

Best Parameters: {'model__learning_rate': 0.05, 'model__max_depth': 7, 'model__n_estimators': 400}
RMSE: 22.354988568908183
R2: 0.2035069465637207
Acc: 0.7252019386106624
F1: 0.2594688724423161


In [11]:
!pip install imbalanced-learn


In [12]:
# =======================================
# 0. 필요한 라이브러리
# =======================================
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from xgboost import XGBClassifier


# =======================================
# 1. Feature / Label 설정
# =======================================
features = [
    'danceability', 'energy', 'valence', 'loudness', 'tempo',
    'acousticness', 'instrumentalness', 'playlist_genre', 'year'
]

X = df[features]
y = (df['track_popularity'] >= 60).astype(int)   # 인기곡 기준 60점


# =======================================
# 2. Train/Test Split
# =======================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


# =======================================
# 3. ColumnTransformer 전처리
# =======================================
numeric_feats = X.select_dtypes(include=['int64', 'float64']).columns
categorical_feats = ['playlist_genre']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_feats),

        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_feats)
    ]
)


# =======================================
# 4. XGBoost 모델 + SMOTE 파이프라인
# =======================================

neg = (y_train == 0).sum()
pos = (y_train == 1).sum()

pipeline = ImbPipeline(steps=[
    ('preprocess', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('model', XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        scale_pos_weight = neg / pos
    ))
])


# =======================================
# 5. GridSearchCV로 XGBoost 튜닝
# =======================================
params = {
    'model__max_depth': [4, 6, 8],
    'model__learning_rate': [0.05, 0.1],
    'model__n_estimators': [200, 400, 600]
}

grid = GridSearchCV(
    pipeline,
    param_grid=params,
    scoring='f1',
    cv=3,
    n_jobs=-1
)

grid.fit(X_train, y_train)

best_model = grid.best_estimator_

print("Best Parameters:", grid.best_params_)


# =======================================
# 6. 테스트셋에서 threshold 튜닝
# =======================================
proba = best_model.predict_proba(X_test)[:, 1]

thresholds = np.arange(0.1, 0.9, 0.02)

best_f1 = 0
best_t = 0

for t in thresholds:
    pred_t = (proba >= t).astype(int)
    f1 = f1_score(y_test, pred_t)
    if f1 > best_f1:
        best_f1 = f1
        best_t = t


# =======================================
# 7. 최종 성능 출력
# =======================================
final_pred = (proba >= best_t).astype(int)

print("\n=== Final Evaluation ===")
print("Best threshold:", best_t)
print("Final F1:", f1_score(y_test, final_pred))
print("Final Accuracy:", accuracy_score(y_test, final_pred))


Best Parameters: {'model__learning_rate': 0.1, 'model__max_depth': 8, 'model__n_estimators': 600}

=== Final Evaluation ===
Best threshold: 0.6400000000000001
Final F1: 0.6098746482476337
Final Accuracy: 0.7536348949919225
